# AMAL TP5
## Réseaux récurrents LSTM, GRU, autres cellules de mémoire


*Comment traiter et générer des séquences de taille variable?*

*Comment prendre en compte des dépendances à plus long terme (Vanishing, exploding, gradients)*

### 1. Générations de séquences de taille variable

Nous considérons que chaque séquence est une phrase (qui peut se terminer par un point, un point d'interrogation ou un point d'exclamation).

Dès lors il faut:
- employer un symbole spécial qui marque la fin d'une séquence EOS. Lors de l'apprentissage, il faut ajouter ce token à chaque séquence pour apprendre à le prédire.
- padder chaque séquence (ajouter un caractère nul (BLANK) autant de fois qu'il est nécessaire afin que les séquences d'un même batch aient toutes la même longueur, ce caractère devra être ignoré lors de l'apprentissage)

##### Q1.
Dans `testloader.py` la classe `TextDataset` est telle que:
- chaque exemple est une phrase
- la taille du dataset est le nbr de phrases ds le corpus
- la phrase renvoyée est sous forme d'une séquence d'entiers, chaque entier codant pr un caractère (application de la fonciton `string2code`)
Cette classe renvoie des exemples de longueur variable. Il faut préciser au DataLoader de quelle manière regrouper les exemples pr construire un batch. C'est le rôle de l'argument `collate_fn` du constructeur d'un DataLoader qui prend une fonction en paramètre.
Définir une fonction `pad_collate_fn` qui prépare un tenseur batch (de taille `longueur x taille du batch` où la longueur = longeur max de la séquence du batch) à partir d'une liste d'exemples du `TextDataset`: elle doit ajouter le code du symbole EOS à la fin de chaque exemple et padder les séquences avec le code du caractère nul. Exécuter `textloader.py`pr vérifier que tout foncitonne bien

De plus, il faut prendre soin de ne pas inclure le padding lorsqu’on calcule le coût ici le maximum de vraisemblance : pour cela, la solution la plus courante est d’utiliser un masque binaire (0 lorsque le caractère est nul, 1 sinon) qui est  multiplié avec les log-probabilités avant
de les additionner (le paramètre `reduce="none"` dans une fonction de coût
(en particulier, pour la cross entropie) permet d’obtenir le coût pour chaque élément plutôt que la moyenne).


In [69]:
%run textloader.py

Chaîne à code :  C'est. Un. Test.
Shape ok
encodage OK
Token EOS ok
Token BLANK ok
Chaîne décodée :  C'est. Un. Test.


/home/nolwenn/AMAL/TME5/tp5/src/textloader.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


#### Q2.

Créer votre fonction de coût `maskedCrossEntropy(output, target, padcar) ` dans `tp5.py` qui permet de calculer le coût sans prendre en compte les caracètres nuls en
fonction de la sortie obtenue output, la sortie désirée target et le code de caractère
de padding padcar. Vous ferez attention à n’utiliser aucune boucle pour ce calcul.

In [70]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from textloader import *
from generate import *
import torch.nn.functional
from pathlib import Path
from datetime import datetime


In [71]:
def maskedCrossEntropy(output: torch.Tensor, target: torch.LongTensor, padcar: int):
    """
    :param output: Tenseur length x batch x output_dim,
    :param target: Tenseur length x batch
    :param padcar: index du caractere de padding
    """
    #  TODO:  Implémenter maskedCrossEntropy sans aucune boucle, la CrossEntropy qui ne prend pas en compte les caractères de padding.

    # Reshape output and target tensors
    output = output.view(-1, output.size(-1))
    target = target.view(-1)

    # Calcul du coût cross-entropique
    ce_loss = torch.nn.CrossEntropyLoss(reduction = 'none')
    loss = ce_loss(output, target)
    
    # Créer un masque en utilisant le caractère de padding, marking non-padding positions with 1.0 and padding positions with 0.0.
    mask = torch.where(target == padcar, 0, 1)
    
    return torch.sum(loss[mask]) / torch.sum(mask)

In [73]:
# Dummy data
output_dim = 10
length = 5
batch_size = 3
padcar = 0

# Randomly generated tensors for output and target
output = torch.rand(length, batch_size, output_dim)
target = torch.randint(0, output_dim, (length, batch_size), dtype=torch.long)

# Call maskedCrossEntropy
loss = maskedCrossEntropy(output, target, padcar)

print("Loss:", loss.item())

Loss: 2.161106586456299


In [87]:
from tp5 import *
input_dim = len(lettre2id)  # Adjust based on your data
latent_dim = 64  # Adjust based on your preference
output_dim = len(lettre2id) + 1  # Adjust based on your data
model = SimpleRNN(input_dim, latent_dim, output_dim)
model

SimpleRNN(
  (embedding): Embedding(97, 64)
  (rnn): RNN(64, 64, batch_first=True)
  (linear_out): Linear(in_features=64, out_features=98, bias=True)
  (tanh): Tanh()
)

#### Q3. 
Dans `generate.py` implémenter une fonction de génération de façon à générer des séquences jusqu'à rencontrer le caractère EOS (penser à prévoir une taille maximum tout de même). Prévoir de faire de la génération aléatoire dans la distribution obtenue ou déterministe en choisissant le caractère le plus probable à chaque pas de temps.

In [92]:
from generate import *
# Test de generate

generated_sequence = generate_sequence(model, initial_input="<START>", eos_token="<EOS>", device='cpu')
print(generated_sequence)
cleaned_sequence = generated_sequence.replace("<START>", "").replace("<PAD>", "").replace("<EOS>", "")
print(cleaned_sequence)

<START>iGonQLWh**AGXD*P&oz<EOS>
iGonQLWh**AGXD*P&oz


In [96]:
# Test de generate_beam
generated_beam_sequence = generate_beam(model, initial_input="<START>", eos_token="<EOS>", k=3, device='cpu')

print("Generated Beam Sequence:", generated_beam_sequence)

Generated Beam Sequence: <START>NIr#Fe&gFpdyDFeFpdyDxs%TB<PAD>N!&VGe&gFpdFu(SYQMVfozb)n$u(o$us!&nSUGXhEHZ<PAD>NyF<PAD>m*CFu(!ags<PAD>&VXUT)!)n)pG*"F


In [107]:
# Assuming your model instance is called 'model'
alpha_value = 0.7
nucleus_function = p_nucleus(model.linear_out, alpha_value)

# Example state tensor (replace with your actual state tensor)
example_state = torch.randn(1, model.rnn.hidden_size)

nucleus_probs = nucleus_function(example_state)
print("Nucleus Sampling Probabilities:", nucleus_probs)


Nucleus Sampling Probabilities: tensor([[0.0000, 0.0000, 0.0231, 0.0108, 0.0203, 0.0109, 0.0145, 0.0148, 0.0000,
         0.0105, 0.0096, 0.0000, 0.0000, 0.0153, 0.0000, 0.0237, 0.0202, 0.0111,
         0.0103, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0094, 0.0097, 0.0190, 0.0183, 0.0000, 0.0107, 0.0098,
         0.0000, 0.0118, 0.0000, 0.0105, 0.0000, 0.0226, 0.0000, 0.0436, 0.0000,
         0.0103, 0.0000, 0.0000, 0.0184, 0.0137, 0.0000, 0.0000, 0.0000, 0.0157,
         0.0181, 0.0000, 0.0000, 0.0200, 0.0000, 0.0000, 0.0000, 0.0085, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0147, 0.0000, 0.0000, 0.0119, 0.0000, 0.0000,
         0.0243, 0.0000, 0.0095, 0.0000, 0.0000, 0.0000, 0.0000, 0.0201, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0145, 0.0149, 0.0000, 0.0180, 0.0205, 0.0000,
         0.0127, 0.0122, 0.0144, 0.0000, 0.0189, 0.0116, 0.0000, 0.0098]],
       grad_fn=<CopySlices>)


Lors de TP4, on a utilisé un encodage onehot pr chaque caractère, suivi d'un module linéaire. Le module `nn.Embedding` de `Torch` permet de combiner ces 2 étapes pr éviter la création (non nécessaire et coûteuse) de vecteurs one hot

### 2. Prise en compte de dépendences lointaines: LSTM et GRU

### 3. Beam Search